# Training a single neuron

## Import Libraries

In [37]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Import Dataset

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [39]:
df.shape

(569, 33)

## Data Processing

In [40]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

In [41]:
df.shape

(569, 31)

## Train Test Split

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42)

In [43]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(455, 30)
(455,)
(114, 30)
(114,)


## Scaling 

In [44]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [45]:
X_train

array([[-1.44075296, -0.43531947, -1.36208497, ...,  0.9320124 ,
         2.09724217,  1.88645014],
       [ 1.97409619,  1.73302577,  2.09167167, ...,  2.6989469 ,
         1.89116053,  2.49783848],
       [-1.39998202, -1.24962228, -1.34520926, ..., -0.97023893,
         0.59760192,  0.0578942 ],
       ...,
       [ 0.04880192, -0.55500086, -0.06512547, ..., -1.23903365,
        -0.70863864, -1.27145475],
       [-0.03896885,  0.10207345, -0.03137406, ...,  1.05001236,
         0.43432185,  1.21336207],
       [-0.54860557,  0.31327591, -0.60350155, ..., -0.61102866,
        -0.3345212 , -0.84628745]])

In [46]:
y_train[:10]

68     B
181    M
63     B
248    B
60     B
15     M
290    B
137    B
155    B
517    M
Name: diagnosis, dtype: object

## Label Encoding : Traget

In [47]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)

In [48]:
y_train[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [49]:
X_train.shape

(455, 30)

## Numpy array to Pytorch Tensors

In [50]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

In [51]:
type(X_test)

torch.Tensor

### Dataset & DataLoader

In [74]:
from torch.utils.data import Dataset, DataLoader

In [75]:
class CustomDataset(Dataset):

    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):

        return self.features.shape[0]

    def __getitem__(self, index):

        return self.features[index], self.labels[index]

In [77]:
# create object of CustomDataset
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [79]:
train_dataset[10]

(tensor([-0.4976,  0.6137, -0.4981, -0.5310, -0.5769, -0.1749, -0.3622, -0.2849,
          0.4335,  0.1782, -0.3684,  0.5531, -0.3167, -0.4052,  0.0403, -0.0380,
         -0.1804,  0.1648, -0.1217,  0.2308, -0.5004,  0.8194, -0.4692, -0.5331,
         -0.0491, -0.0416, -0.1491,  0.0968,  0.1062,  0.4904]),
 tensor(0.))

In [80]:
# create DataLoader object
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True)

## Define Model

In [68]:
class MySimpleNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
        
    # forward pass
    def forward(self, features):
        out = self.linear(features)
        out = self.sigmoid(out)
        return out
    

## Important Parameters

In [108]:
learning_rate = 0.1
epochs = 25

In [69]:
# define loss function
loss_function = nn.BCELoss()

In [81]:
# create model
model = MySimpleNN(X_train.shape[1])

In [82]:
# define optimizer:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Training Pipeline

In [110]:
# define loop
for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:
    
        # forward pass
        y_pred = model(X_train)
    
        # loss calculation
        loss = loss_function(y_pred, y_train.view(-1,1))
    
        # zeros gradients
        optimizer.zero_grad()
        
        # backward pass
        loss.backward()
    
        # parameter update
        optimizer.step()

    # if epoch % 10 == 0:
    # print loss in each epoch
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.026280216872692108
Epoch 2, Loss: 0.026262106373906136
Epoch 3, Loss: 0.02624402567744255
Epoch 4, Loss: 0.026225995272397995
Epoch 5, Loss: 0.026208050549030304
Epoch 6, Loss: 0.02619014121592045
Epoch 7, Loss: 0.026172269135713577
Epoch 8, Loss: 0.026154428720474243
Epoch 9, Loss: 0.026136599481105804
Epoch 10, Loss: 0.026118841022253036
Epoch 11, Loss: 0.02610113099217415
Epoch 12, Loss: 0.026083458214998245
Epoch 13, Loss: 0.026065822690725327
Epoch 14, Loss: 0.026048215106129646
Epoch 15, Loss: 0.0260306429117918
Epoch 16, Loss: 0.026013119146227837
Epoch 17, Loss: 0.02599567174911499
Epoch 18, Loss: 0.02597825974225998
Epoch 19, Loss: 0.025960899889469147
Epoch 20, Loss: 0.025943582877516747
Epoch 21, Loss: 0.025926323607563972
Epoch 22, Loss: 0.025909096002578735
Epoch 23, Loss: 0.025891877710819244
Epoch 24, Loss: 0.02587468922138214
Epoch 25, Loss: 0.025857537984848022


In [103]:
print("Weights shape", model.linear.weight.shape)
print("Bias shape", model.linear.bias.shape)

Weights shape torch.Size([1, 30])
Bias shape torch.Size([1])


In [104]:
print(model.linear.weight)

Parameter containing:
tensor([[-0.4010,  0.1108, -0.9944, -0.2805, -0.1015, -5.7918,  2.6952,  5.9848,
         -1.4980,  2.4729,  6.4094, -0.7585, -1.1282,  4.1857,  1.1878,  0.4942,
         -3.9775,  2.6538, -2.2356, -2.7442,  2.8401,  3.3918, -0.4880,  3.1462,
         -0.5830, -0.0812,  4.8254,  0.5405,  4.4000, -0.3354]],
       requires_grad=True)


## Evaluation

In [105]:
X_test = X_test.float()  # Convert input to Float tensor
y_test = y_test.float()  # Convert labels to Float tensor

In [106]:
# Set the model to evaluation mode
model.eval()

accuracy_list = []

with torch.no_grad():
    
    for batch_features, batch_labels in test_loader:
        # forward pass
        y_pred = model.forward(batch_features)
        # convert in 1 or 0
        y_pred = (y_pred > 0.5).float()
        # calculate accuracy for cuurent batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)


## Accuracy

In [107]:
# calculate overall accuracy
accuracy = sum(accuracy_list) / len(accuracy_list)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9561
